<a href="https://colab.research.google.com/github/gabrielborja/parc_de_montjuic/blob/main/interactive_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Orders Visualization

In [ ]:
# Importing python libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import plotly.express as px
from ipywidgets import interact

## Loading Data

In [ ]:
# Loading data from local drive
from google.colab import files
uploaded1 = files.upload()

In [ ]:
# Storing loaded data from excel to a pandas dataframe
import io
df1 = pd.read_excel(io.BytesIO(uploaded1['Script_202209080812.xlsx']))

In [ ]:
# Chainging columns to lower case
df1.rename(columns={i:i.lower() for i in df1.columns}, inplace=True)

In [ ]:
# Checking the dataframe info
df1.info()

## Visualization

In [ ]:
# Exploring list of columns with value counts
@interact(Column_name=df1.columns, Percentage=[True, False])
def explore_value_counts(Column_name, Percentage):
  df = df1.value_counts(subset=[Column_name], normalize=Percentage, dropna=False)
  return df

In [ ]:
# Creating categories for interactive visualization
cat_list_1 = df1.loc[:,'product_family'].unique().tolist()
time_list_1 = ['order_date','activation_date','cancellation_date']

In [ ]:
# Creating function to aggregate order dates
def aggregate_order_dates():
  df_list = []
  for i in cat_list_1:
    for j in time_list_1:
      df = df1[df1['product_family']==i].copy().reset_index(drop=True)
      df = df.groupby(by=[j]).agg(total=('order_no','count')).reset_index()
      df = df.rename(columns={j:'period'})
      df = df.assign(category=i, order_type=j, cum_sum=df['total'].cumsum())
      df_list.append(df)
  df_con = pd.concat(df_list, axis=0) #=> Stacking df on top of each others
  return df_con

df1_or = aggregate_order_dates()

In [ ]:
# Exploring feature importance in same graph
@interact(Category=cat_list_1)
def plot_feature(Category):
  ''' Function for comparing product family orders agains time '''
  df = df1_or[df1_or['category']==Category].copy().reset_index(drop=True)
  fig_x, axes = plt.subplots(1, 1, figsize=(7.5, 7.5))
  axes.set_title(f'{Category}: 50K Hunt')
  sns.lineplot(data=df, x='period', y='cum_sum', hue='order_type')#,ax=axes)
  for container in axes.containers:
    axes.bar_label(container, fontsize=12)
  #fig_x.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  plt.xticks(rotation=30)
  plt.show()